In [1]:
from jupyter_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import os
import numpy as np
import pandas as pd
import base64

# Replace with your actual CRUD module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# MongoDB credentials and connection
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)

# Retrieve data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Add Grazioso Salvare logo
image_filename = 'Grazioso_Salvare_Logo.png'  # Replace with your own image path
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App layout
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Project Two Dashboard - EP'))),
    html.Center(html.P('Select Options')),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DRIT'},
                {'label': 'Reset - returns unfiltered state', 'value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Define filter conditions based on rescue type
    if filter_type == 'WR':
        query = {'$and': [{'sex_upon_outcome': 'Intact Female'},
                          {'$or': [
                              {'breed': 'Labrador Retriever Mix'},
                              {'breed': 'Chesa Bay Retr Mix'},
                              {'breed': 'Newfoundland Mix'},
                              {'breed': 'Newfoundland/Labrador Retriever'},
                              {'breed': 'Newfoundland/Australian Cattle Dog'},
                              {'breed': 'Newfoundland/Great Pyrenees'}]},
                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]}]}
    elif filter_type == 'MWR':
        query = {'$and': [{'sex_upon_outcome': 'Intact Male'},
                          {'$or': [
                              {'breed': 'German Shepherd'},
                              {'breed': 'Alaskan Malamute'},
                              {'breed': 'Old English Sheepdog'},
                              {'breed': 'Rottweiler'},
                              {'breed': 'Siberian Husky'}]},
                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]}]}
    elif filter_type == 'DRIT':
        query = {'$and': [{'sex_upon_outcome': 'Intact Male'},
                          {'$or': [
                              {'breed': 'Doberman Pinscher'},
                              {'breed': 'German Shepherd'},
                              {'breed': 'Golden Retriever'},
                              {'breed': 'Bloodhound'},
                              {'breed': 'Rottweiler'}]},
                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]}]}
    else:
        query = {}

    # Retrieve filtered data from MongoDB
    df = pd.DataFrame.from_records(shelter.read(query))

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')

    return data, columns

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Function to update the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [html.P("No data to display")]

    fig = px.pie(
        data_frame=dff, 
        names='breed', 
        title='Distribution of Breeds in Filtered Data',
        color_discrete_sequence=px.colors.sequential.RdBu
    )
    
    return [dcc.Graph(figure=fig)]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return [html.P("No data to display")]

    if not selected_rows:
        selected_rows = list(range(len(dff)))

    markers = [
        dl.Marker(position=[dff.iloc[i]['location_lat'], dff.iloc[i]['location_long']], children=[
            dl.Tooltip(dff.iloc[i]['breed']),
            dl.Popup([
                html.H4("Animal Name"),
                html.P(dff.iloc[i]['name']),
                html.H4("Sex"),
                html.P(dff.iloc[i]['sex_upon_outcome']),
                html.H4("Breed"),
                html.P(dff.iloc[i]['breed']),
                html.H4("Age"),
                html.P(dff.iloc[i]['age_upon_outcome'])
            ])
        ]) for i in selected_rows
    ]

    return [
        dl.Map(style={'width': '500px', 'height': '250px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            *markers
        ])
    ]

if __name__ == '__main__':
    app.run_server(debug=True)


/tmp/ipykernel_1043114/2554075587.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipykernel_1043114/2554075587.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/ipykernel_1043114/2554075587.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


Dash app running on http://127.0.0.1:27275/
